In [1]:
import sys

import importlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sqlalchemy
import gensim
import logging

import common
import util
importlib.reload(common)
importlib.reload(util)

from common import create_engine
from common import display_all
from common import figsize
from common import save_df, load_df
from common import save_session, load_session

from util import show_importances
from util import split_X_y_all, split_X_y, split_data
from util import empty_features, column_feature, str_contains

from pbar import Pbar

from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters() # converters e.g. for datetime in plots
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [2]:
RANDOM_STATE = 123
np_random = np.random.RandomState(RANDOM_STATE)

In [3]:
df = load_df('final_data.pickle')

--

1. popularita clankov (oktober vs dnes)
2. analyzovat rozlozenie popularity v dnoch
3. pozriet sa na pocty pre reliable a unreliable

1. pocet clankov
2. zdroje
3. 